<a href="https://colab.research.google.com/github/liuyixin-louis/dl_learning/blob/master/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人民日报+无预训练词嵌入+bilstm(筛后样本估计太小，f1很高，但是实际效果很差)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

input (InputLayer)           [(None, 97)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 97, 100)           350000    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 97, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 97, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 97, 4)             1028      
_________________________________________________________________
activation_1 (Activation)    (None, 97, 4)             0         

In [13]:
!pip install 'kashgari>=1.0.0,<2.0.0'

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 17.3MB 207kB/s 
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=715b34d69c6943a960a37f7d5693be74c195caa413bc6879c369ad159137be88
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=a461c14f3a992d0d6ae63e11826febbbbb0f8fb87efe573faebae3defc8baba1
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for bert4keras: filename=bert4keras-0.6.5-cp36-none-any.whl size=34212 sha256=70c2889023eee6449b3ec9f4f9b7c5bdcadc9e4252193eb4c37b3c324e753a1c
  Stored in directory: /root/.cache/pip/wheels/97/82/72/56a894ccb2337a25b679cbac552bf2f15f5e8e798a37313de6
  Created wheel for keras-transformer: filename=keras_transformer-0.33.0-cp36-none-any.w

In [14]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [15]:
from kashgari.corpus import ChineseDailyNerCorpus
from kashgari.tasks.labeling import BiLSTM_Model

In [16]:
import kashgari
kashgari.config.use_cudnn_cell = True

In [0]:
train_x, train_y = ChineseDailyNerCorpus.load_data('train')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')

In [47]:
test_y[12][5] == 'I-LOC'

True

In [0]:
def only_location(y):
  for yi in y:
    for i in range(len(yi)):
      label =  yi[i]
      if label == 'B-LOC' or label == 'I-LOC':
        pass
      else:
        yi[i] = 'O'
only_location(test_y)
only_location(valid_y)
only_location(train_y)

In [0]:
test_y

In [58]:
k = 0
for i in train_y:
  if 'B-LOC' in i:
    k+=1
k

7747

In [51]:
model = BiLSTM_Model()
model.fit(train_x, train_y, valid_x, valid_y, epochs=50)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 97)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 97, 100)           350000    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 97, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 97, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 97, 4)             1028      
_________________________________________________________________
activation_1 (Activation)    (None, 97, 4)             0         
Total params: 586,548
Trainable params: 586,548
Non-trainable params: 0
_____________________________________________________

KeyboardInterrupt: ignored

In [0]:
def makedata(s):
  res = []
  for i in s:
    res.append(i)
  return [res]

def visualizeResult(x,y):
  for xi,yi in zip(x,y):
    for i in range(len(xi)):
      if yi[i]!='O':
        print (xi[i]+yi[i])

In [0]:
# x_data = makedata('疫情通报：。番禺区：小明在确诊病症后，居住于华南理工大学C10学生宿舍，活动于学生第一食堂。')
x_data = makedata('小明住在广东省深圳市宝安区西乡街道，确诊之后经常活动于好又多超市')
x_data

In [0]:
y = model.predict(x_data)

In [74]:
visualizeResult(x_data,y)

广B-LOC
东I-LOC
省I-LOC
深B-LOC
圳I-LOC
市I-LOC
宝B-LOC
安I-LOC
区I-LOC
西I-LOC
乡I-LOC


# CLUENER2020+无预训练词嵌入+bilstm+crf（50epoches,best f1:0.6113左右）


In [0]:
# self.emebdding_size = embedding_size
# self.embedding = nn.Embedding(vocab_size, embedding_size)
# self.bilstm = nn.LSTM(input_size=embedding_size,hidden_size=hidden_size,
#                       batch_first=True,num_layers=2,dropout=drop_p,
#                       bidirectional=True)
# self.dropout = SpatialDropout(drop_p)
# self.layer_norm = LayerNorm(hidden_size * 2)
# self.classifier = nn.Linear(hidden_size * 2,len(label2id))
# self.crf = CRF(tagset_size=len(label2id), tag_dictionary=label2id, device=device)

In [0]:
!wget https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip

--2020-05-05 07:10:04--  https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1054686 (1.0M) [application/zip]
Saving to: ‘cluener_public.zip’

cluener_public.zip  100%[===================>]   1.00M  --.-KB/s    in 0.009s  

2020-05-05 07:10:04 (108 MB/s) - ‘cluener_public.zip’ saved [1054686/1054686]



In [0]:
! unzip /content/cluener_public.zip -d ./cluener_public/

Archive:  /content/cluener_public.zip
  inflating: ./cluener_public/train.json  
  inflating: ./cluener_public/dev.json  
  inflating: ./cluener_public/test.json  
  inflating: ./cluener_public/cluener_predict.json  
  inflating: ./cluener_public/README.md  


In [0]:
! git clone  https://github.com/CLUEbenchmark/CLUENER2020/

Cloning into 'CLUENER2020'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 352 (delta 0), reused 15 (delta 0), pack-reused 333
Receiving objects: 100% (352/352), 890.81 KiB | 6.70 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [0]:
!pip install jsonlines

In [0]:

import os
 
# print(os.getcwd())
 
 
# 修改工作路径
# import os
# from google.colab import drive
# drive.mount('/content/gdrive')
 
path = "/content/CLUENER2020/bilstm_crf_pytorch"
os.chdir(path)

In [0]:
!python download_clue_data.py --data_dir=./dataset --tasks=cluener

	Completed! Downloaded cluener data to directory ./dataset/cluener


In [0]:
!python run_lstm_crf.py --do_train

05/05/2020 07:15:49 - INFO - root -   Loading features from cached file dataset/cluener/cached_crf-train_bilstm_crf_ner
66 batches created
Epoch 1/50
[Training] 66/66 [==============================] 192.7ms/step  loss: 3.4617  
05/05/2020 07:16:02 - INFO - root -   Loading features from cached file dataset/cluener/cached_crf-dev_bilstm_crf_ner
9 batches created
[Evaluating] 9/9 [==============================] 763.1ms/step 
05/05/2020 07:16:09 - INFO - root -   
Epoch: 1 -  loss: 15.1076 - eval_loss: 11.3870 - eval_acc: 0.3220 - eval_recall: 0.3056 - eval_f1: 0.3136 
05/05/2020 07:16:09 - INFO - root -   
Epoch 1: eval_f1 improved from 0 to 0.3136176066024759
05/05/2020 07:16:09 - INFO - root -   save model to disk.
Eval Entity Score: 
05/05/2020 07:16:09 - INFO - root -   Subject: address - Acc: 0.322 - Recall: 0.3056 - F1: 0.3136
Epoch 2/50
[Training] 66/66 [==============================] 198.2ms/step  loss: 1.8282  
05/05/2020 07:16:22 - INFO - root -   Loading features from cache

In [0]:
!python run_lstm_crf.py --do_eval

05/05/2020 07:32:22 - INFO - root -   loading model from outputs/bilstm_crf/best-model.bin .
05/05/2020 07:32:22 - INFO - root -   Loading features from cached file dataset/cluener/cached_crf-dev_bilstm_crf_ner
9 batches created
[Evaluating] 9/9 [==============================] 759.0ms/step 


In [0]:
!python run_lstm_crf.py --do_predict

05/05/2020 07:32:34 - INFO - root -   loading model from outputs/bilstm_crf/best-model.bin .
[Training] 10632/10632 [==============================] 34.6ms/step 
